# CHAPTER 4 조인 · 병합 · 변형

## 목표
- 여러 테이블을 키로 연결(merge)해 사실테이블(fact)을 만든다.
- pivot_table로 보고서형 데이터(wide)를 만든다.


In [1]:
# ============================================
# [공통] 라이브러리 / 경로 / 출력 옵션 세팅
# 이 셀은 모든 챕터 노트북에서 동일하게 사용합니다.
# ============================================

# 1) 수치 계산(NumPy) / 표 데이터(Pandas) 불러오기
import numpy as np
import pandas as pd

# 2) 파일 경로를 운영체제와 무관하게 다루기 위한 Path
from pathlib import Path

# 3) 현재 노트북이 실행되는 폴더를 기준(BASE)으로 데이터 폴더(DATA) 지정
BASE = Path(".").resolve()          # 현재 작업 폴더(절대경로)
DATA = BASE / "data"                # data 폴더 경로

# 4) Pandas 출력 옵션(교육용): 너무 길게 출력되지 않도록 적당히 제한
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 140)

# 5) 확인 출력
print("BASE:", BASE)
print("DATA exists:", DATA.exists())


BASE: C:\Users\KDA\python_Code
DATA exists: True


## 4.1 데이터 로드

In [2]:
# [이 셀은 무엇을 하는가?]
# - 조인 실습용 테이블 4개를 로드합니다.

customers = pd.read_csv(DATA / "customers.csv", parse_dates=["signup_date"])
orders    = pd.read_csv(DATA / "orders.csv", parse_dates=["order_date"])
items     = pd.read_csv(DATA / "order_items.csv")
products  = pd.read_csv(DATA / "products.csv")

print("customers:", customers.shape)
print("orders   :", orders.shape)
print("items    :", items.shape)
print("products :", products.shape)

customers: (30, 4)
orders   : (120, 4)
items    : (298, 3)
products : (15, 3)


## 4.2 고객 + 주문 merge

In [3]:
# [이 셀은 무엇을 하는가?]
# - 주문 테이블에 고객의 region/age를 붙입니다(LEFT JOIN).

orders_c = orders.merge(
    customers[["customer_id","region","age"]],
    on="customer_id",
    how="left"
)

display(orders_c.head())

,order_id,customer_id,order_date,channel,region,age
0,O00001,C0021,2024-01-15,Web,Incheon,58
1,O00002,C0016,2024-01-15,Web,Incheon,47
2,O00003,C0022,2024-01-28,App,Daegu,43
3,O00004,C0023,2024-01-09,App,Daegu,44
4,O00005,C0028,2024-02-21,Web,Incheon,21


## 4.3 주문상세 + 상품 + 주문 merge → fact 테이블

In [4]:
# [이 셀은 무엇을 하는가?]
# - order_items에 products를 붙인 뒤
# - orders_c를 붙여 최종 fact 테이블을 만들고 amount(매출)를 계산합니다.

items_p = items.merge(products, on="product_id", how="left")
fact = items_p.merge(orders_c, on="order_id", how="left")

fact["amount"] = fact["qty"] * fact["unit_price"]         # 파생지표: 매출  # 있는 컬럼, 없는 컬럼에 따라서 값이 들어가면 오른쪽 끝에 들어가고 , 결과값 출력 
display(fact.head())
print("fact.shape:", fact.shape)

,order_id,product_id,qty,category,unit_price,customer_id,order_date,channel,region,age,amount
0,O00001,P005,1,Book,12128,C0021,2024-01-15,Web,Incheon,58,12128
1,O00001,P011,2,Clothing,47371,C0021,2024-01-15,Web,Incheon,58,94742
2,O00002,P011,3,Clothing,47371,C0016,2024-01-15,Web,Incheon,47,142113
3,O00002,P013,2,Clothing,47932,C0016,2024-01-15,Web,Incheon,47,95864
4,O00002,P014,2,Clothing,40694,C0016,2024-01-15,Web,Incheon,47,81388


fact.shape: (298, 11)


## 4.4 pivot_table: 일자×카테고리 매출표

In [5]:
# [이 셀은 무엇을 하는가?]
# - 보고서 형태(날짜 행, 카테고리 열)의 매출표를 만듭니다.

daily_cat = fact.pivot_table(
    index="order_date",
    columns="category",
    values="amount",
    aggfunc="sum",
    fill_value=0
).sort_index()

display(daily_cat.head())

out_path = DATA / "daily_category_sales.csv"
daily_cat.to_csv(out_path)
print("saved:", out_path)

category,Book,Clothing,Food,Gadget
order_date,,,,
2024-01-01,357966,194942,142321,104750
2024-01-03,177417,340696,72009,204848
2024-01-04,0,503801,59159,17846
2024-01-05,0,147765,0,0
2024-01-06,0,258631,24003,0


saved: C:\Users\KDA\python_Code\data\daily_category_sales.csv
